`TCP` and `UDP` constitute(تشکیل می دهند) the ***transport layer protocols*** on top of which most internet— and LAN—services are built.

***HTTP*** (version 2 and below), ***FTP***, and ***SMTP*** use `TCP`   
***DNS*** and ***HTTP version 3*** use `UDP`.

***TCP*** is `connection-oriented` and includes `reliability mechanisms`   
***UDP*** is `connectionless`, has a **lower overhead**, and **supports broadcasting**.   
  
`BitTorrent` uses ***UDP***, as does Voice over IP (`VoIP`).

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h5>کلاس‌های TcpClient و TcpListener:</h5>
<ul><li><strong>TcpClient و TcpListener</strong>: این دو کلاس در .NET برای کار با پروتکل TCP استفاده می‌شوند و به عنوان یک واسط ساده‌تر (façade) برای مدیریت ارتباطات TCP عمل می‌کنند.</li><li><strong>سادگی استفاده</strong>: این کلاس‌ها به گونه‌ای طراحی شده‌اند که استفاده از آنها نسبتاً ساده است و بسیاری از جزئیات پیچیده‌تر ارتباطات شبکه را پنهان می‌کنند.</li></ul>
<h5>کلاس Socket:</h5>
<ul><li><strong>Socket</strong>: این کلاس دارای قابلیت‌های بیشتری نسبت به TcpClient و TcpListener است و امکان پیکربندی پیشرفته‌تر و دسترسی مستقیم به لایه شبکه (IP) و پروتکل‌های غیر اینترنتی مانند SPX/IPX نوول را فراهم می‌کند.</li><li><strong>پیکربندی پیشرفته</strong>: کلاس Socket گزینه‌های بیشتری برای پیکربندی ارتباطات شبکه فراهم می‌کند، که می‌تواند برای برنامه‌های پیچیده‌تر مفید باشد.</li><li><strong>دسترسی به لایه شبکه</strong>: با استفاده از این کلاس، می‌توانید به لایه‌های پایین‌تر شبکه دسترسی پیدا کنید و حتی از پروتکل‌های شبکه‌ای غیر از TCP/IP استفاده کنید.</li></ul>

<h3>مخلوط کردن و مطابقت:</h3>
<ul><li><strong>مخلوط کردن و مطابقت</strong>: می‌توانید از ترکیب این کلاس‌ها استفاده کنید. به عنوان مثال، کلاس TcpClient یک ویژگی به نام <code>Client</code> دارد که به شما اجازه می‌دهد به شیء زیرین Socket دسترسی پیدا کنید و از قابلیت‌های پیشرفته‌تر آن استفاده کنید.</li></ul>
<p>به طور خلاصه، اگر به دنبال یک راه ساده برای مدیریت ارتباطات TCP هستید، می‌توانید از TcpClient و TcpListener استفاده کنید. اما اگر به قابلیت‌های پیشرفته‌تری نیاز دارید، کلاس Socket گزینه بهتری خواهد بود.</p>
</div>

In [ ]:
//the basic structure for a synchronous TCP client request

using System.Net.Sockets;

using (TcpClient client = new TcpClient())
{
    client.Connect ("address", 8880);
    using (NetworkStream n = client.GetStream())
    {
        // Read and write to the network stream...
    }
}

***TcpClient’s Connect*** method `blocks` until a connection is ***established*** (Connect Async is the asynchronous equivalent).   
The ***NetworkStream*** then provides a means of `two-way communication`, for both **transmitting** and **receiving** `bytes` of data from a server.

In [ ]:
//A simple TCP server looks like this:

using System.Net.Sockets;

TcpListener listener = new TcpListener (new IPAddress(""), 8880);
listener.Start();

while (true)
    using (TcpClient c = listener.AcceptTcpClient())
    using (NetworkStream n = c.GetStream())
    {
        // Read and write to the network stream...
    }
listener.Stop();

In [1]:
using System;
using System.IO;
using System.Net;
using System.Net.Sockets;
using System.Threading;

new Thread(Server).Start(); // Run server method concurrently.
Thread.Sleep(500); // Give server time to start.
Client();
void Client()
{
    using (TcpClient client = new TcpClient("localhost", 51111))
    using (NetworkStream n = client.GetStream())
    {
        BinaryWriter w = new BinaryWriter(n);
        w.Write("Hello");
        w.Flush();
        Console.WriteLine(new BinaryReader(n).ReadString());
    }
}
void Server() // Handles a single client request, then exits.
{
    TcpListener listener = new TcpListener(IPAddress.Any, 51111);
    listener.Start();
    using (TcpClient c = listener.AcceptTcpClient())
    using (NetworkStream n = c.GetStream())
    {
        string msg = new BinaryReader(n).ReadString();
        BinaryWriter w = new BinaryWriter(n);
        w.Write(msg + " right back!");
        w.Flush(); // Must call Flush because we're not
    } // disposing the writer.
    listener.Stop();
}
// OUTPUT: Hello right back!

Hello right back!


### Concurrency with TCP

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<ol><li><p><strong>تبدیل روش‌های مسدودکننده به ناهمزمان</strong>: به جای استفاده از روش‌های سنتی مسدودکننده، از نسخه‌های <code>*Async</code> این متدها استفاده کنید. مثلاً به جای استفاده از <code>AcceptTcpClient()</code>، از <code>AcceptTcpClientAsync()</code> استفاده می‌کنید.</p></li><li><p><strong>استفاده از <code>await</code></strong>: برای صبر کردن (await) بر روی task (کار) که توسط این متدهای ناهمزمان بازگشت داده می‌شود، از کلمه کلیدی <code>await</code> استفاده کنید. این کار به شما اجازه می‌دهد که بدون مسدود کردن thread اصلی، به اجرای بقیه کد ادامه دهید تا زمانی که عملیات ناهمزمان به پایان برسد.</p></li></ol>

</div>

In [ ]:
using System;
using System.Net.Sockets;
using System.Threading.Tasks;

async void RunServerAsync ()
{
    TcpListener listener = new TcpListener(System.Net.IPAddress.Any, 8000);
    listener.Start();

    while (true)
    {
        TcpClient client = await listener.AcceptTcpClientAsync();
        HandleClientAsync(client);
    }
}

static async Task HandleClientAsync(TcpClient client)
{
    await Task.Yield ();
    try
    {
        using (client)
        using (NetworkStream n = client.GetStream ())
        {
            byte[] data = new byte [5000];
            
            int bytesRead = 0; int chunkSize = 1;
            while (bytesRead < data.Length && chunkSize > 0)
                bytesRead += chunkSize =
                    await n.ReadAsync (data, bytesRead, data.Length - bytesRead);

            Array.Reverse (data); // Reverse the byte sequence
            await n.WriteAsync (data, 0, data.Length);
        }
    }
    catch (Exception ex) { Console.WriteLine (ex.Message); }
}